In [66]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from multiprocessing import Pool, Lock, Value
from time import sleep
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import base64
import numpy as np

In [11]:
df = pd.read_csv('data/kaggle_train_data_tab_new.csv', sep='\t')

In [12]:
df.head()

,Id,Prediction,Url,PageBase64
0,-9222401963271173253,0,http://lawleader.ru/docs/32/,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...
1,-9221399504663603656,0,http://alterhost.ruwww.alterhost.ruwww.user.kz...,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...
2,-9219733223042265364,0,http://www.ferio.ru/c_1-acura-legend.html,PCFET0NUWVBFIEhUTUwgUFVCTElDICItLy9XM0MvL0RURC...
3,-9218561774155397294,0,http://fshops.otzyv.ru/read.php?id=277,PCFET0NUWVBFIEhUTUwgUFVCTElDICItLy9XM0MvL0RURC...
4,-9216163564171210203,1,http://penhea895.narod.ru/familiya-shushmarche...,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...


In [17]:
df['Prediction'].values

array([0, 0, 0, ..., 1, 1, 0])

In [58]:
def extract_features(html_page):
    html = base64.b64decode(html_page)
    soup = BeautifulSoup(html)
    if soup.title:
        title_words_cnt = len(soup.title.text.split())
    else:
        title_words_cnt = 0
    if soup.text:
        words_cnt = len(soup.text.split())
    else:
        words_cnt = 0
    link_cnt = len(soup.find_all('link'))
    
    return [title_words_cnt, words_cnt, link_cnt]

In [38]:
html = base64.b64decode(df['PageBase64'].values[0])
soup = BeautifulSoup(html)

In [46]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Договор займа, договоры, договора</title>
<meta content="договоры, договора" name="keywords"/>
<meta content="договоры, договора" name="description"/>
<meta content="help-pro.ru" name="author"/>
<link href="/style.css" rel="stylesheet" type="text/css"/>
<link href="/css/flick/jquery-ui-1.8.2.custom.css" rel="stylesheet" type="text/css"/>
<link href="/balloontip.css" rel="stylesheet" type="text/css"/>
<link href="http://lawleader.ru/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script src="/inc/jquery.js" type="text/javascript"></script>
<script src="/inc/jquery-ui.js" type="text/javascript"></script>
<script src="/balloontip.js" type="text/javascript"></script>
<script type="text/JavaScript">
		<!--
		function poll(answer,question

In [59]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(page):
    res = extract_features(page) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, df['PageBase64'].values)


3070 objects are processed...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


7040 objects are processed...

In [64]:
X_train = np.array(res)
y_train = df['Prediction'].values

In [68]:
clf = XGBClassifier()
scores = cross_validate(clf, X_train, y_train, cv=3, scoring='f1')

In [73]:
scores['test_score'].mean()

0.8036788871758325